In [1]:
from pathlib import Path
from multiprocessing import Pool
import time
from tqdm.notebook import tqdm
from stexls.vscode import *
from stexls.util.workspace import Workspace
from stexls.stex.compiler import Compiler
from stexls.stex.linker import Linker
from stexls.linter.linter import Linter
from stexls.stex.references import *
import os, sys

In [2]:
outdir = Path('/tmp/out')
mathhub = Path('~/MathHub').expanduser()
agenttypes = Path('/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/agent-types.tex')
balancedprime = Path('/home/marian/MathHub/smglom/primes/source/balancedprime.en.tex')
assert mathhub.is_dir()
ws = Workspace(mathhub)
ws.include = '.*/smglom/.*'
print(len(ws.files))
c = Compiler(ws.root, outdir)
ln = Linker(outdir)
global_step = False
linter = Linter(ws, outdir, enable_global_validation=global_step, num_jobs=8)
if global_step:
    it = list(tqdm(linter.compile_workspace()))

2951


In [3]:
res = linter.lint(balancedprime)

In [5]:
res.object.get_definitions_at(Position(2, 47))

[[Location uri="file:///home/marian/MathHub/smglom/primes/source/primenumber.tex" range=[Range (0 31) (0 44)]],
 [Location uri="file:///home/marian/MathHub/smglom/primes/source/primenumber.tex" range=[Range (2 1) (2 42)]],
 [Location uri="file:///home/marian/MathHub/smglom/primes/source/primenumber.tex" range=[Range (3 1) (3 51)]]]